# 4.2 进阶 Prompt 技巧

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/04-prompt-engineering/advanced_techniques.ipynb)

**预计 API 费用：~$0.02**

本 Notebook 演示 Few-shot、Chain-of-Thought、Self-Consistency 等进阶技巧。

In [ ]:
!pip install -q openai

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("请输入你的 OpenAI API Key: ")

## 实验 1：Zero-shot vs Few-shot 对比

对比没有示例（Zero-shot）和给出示例（Few-shot）的效果差异。

**任务：情感分类**

In [ ]:
from openai import OpenAI

client = OpenAI()

# 测试用例（边界情况）
test_cases = [
    "这个产品太棒了！用了一周非常满意。",
    "质量太差了，用了两天就坏了，非常失望。",
    "还行吧，凑合用，没什么特别的。",
    "价格有点贵，但质量确实不错，总体还算满意。",  # 混合情感
    "不推荐购买。",  # 简短
]

print("=== Zero-shot（无示例） ===")
print()

for comment in test_cases:
    prompt = f"判断评论情感（正面/负面/中性）：{comment}"
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # 确定性输出
    )
    
    result = response.choices[0].message.content
    print(f"评论: {comment}")
    print(f"情感: {result}")
    print()

In [ ]:
print("=== Few-shot（有示例） ===")
print()

# Few-shot Prompt 模板
few_shot_template = """
判断评论的情感倾向（正面/负面/中性）。

示例：
评论：这个产品太棒了！用了一周非常满意。
情感：正面

评论：质量太差了，用了两天就坏了，非常失望。
情感：负面

评论：还行吧，凑合用，没什么特别的。
情感：中性

现在判断这条：
评论：{comment}
情感：
"""

for comment in test_cases:
    prompt = few_shot_template.format(comment=comment)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    
    result = response.choices[0].message.content.strip()
    print(f"评论: {comment}")
    print(f"情感: {result}")
    print()

**观察：**
- Zero-shot 在简单情况下表现不错，但遇到边界情况（混合情感、简短文本）容易出错
- Few-shot 通过示例"教会"模型任务的模式，准确率明显提升
- Few-shot 的缺点：消耗更多 token（示例也要付费）

## 实验 2：Chain-of-Thought（思考链）

让 AI "把思考过程写出来"，显著提升数学推理准确率。

In [ ]:
from openai import OpenAI

client = OpenAI()

# 测试：数学问题
math_problems = [
    "一个数乘以 3 再加 7 等于 28，这个数是多少？",
    "小明有 15 个苹果，给了小红 3 个，又买了 8 个，现在有几个？",
    "一件衣服原价 200 元，打 8 折后又减 20 元，最终多少钱？",
]

print("=== 直接提问（容易出错） ===")
print()

for problem in math_problems:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": problem}],
        temperature=0,
    )
    
    result = response.choices[0].message.content
    print(f"问题: {problem}")
    print(f"回答: {result}")
    print()

In [ ]:
print("=== Chain-of-Thought（逐步推理） ===")
print()

for problem in math_problems:
    prompt = f"""
{problem}

请一步步思考：
1. 列出方程或计算步骤
2. 逐步求解
3. 验证答案
"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    
    result = response.choices[0].message.content
    print(f"问题: {problem}")
    print(f"推理过程:\n{result}")
    print("\n" + "="*80 + "\n")

**Zero-shot CoT：只加一句话**

最简单的 CoT 技巧：加一句 "Let's think step by step." 或 "让我们一步步思考。"

In [ ]:
print("=== Zero-shot CoT（魔法咒语） ===")
print()

for problem in math_problems:
    prompt = f"{problem}\n\n让我们一步步思考。"
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    
    result = response.choices[0].message.content
    print(f"问题: {problem}")
    print(f"推理:\n{result}")
    print("\n" + "="*80 + "\n")

**Few-shot CoT：给出推理示例**

最强效果：给出包含推理过程的示例。

In [ ]:
print("=== Few-shot CoT（带推理的示例） ===")
print()

few_shot_cot_template = """
请解决数学问题，并写出推理步骤。

问题：小明有 15 个苹果，给了小红 3 个，又买了 8 个，现在有几个？
推理：
- 初始：15 个
- 给出：15 - 3 = 12 个
- 买入：12 + 8 = 20 个
答案：20 个

问题：一件衣服原价 200 元，打 8 折后又减 20 元，最终多少钱？
推理：
- 打折：200 × 0.8 = 160 元
- 再减：160 - 20 = 140 元
答案：140 元

现在解决这个问题：
问题：{problem}
推理：
"""

test_problem = "一个数乘以 3 再加 7 等于 28，这个数是多少？"
prompt = few_shot_cot_template.format(problem=test_problem)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
)

print(f"问题: {test_problem}")
print(f"\n推理:\n{response.choices[0].message.content}")

**关键洞察：**
- CoT 让模型"展示草稿纸"，减少跳步错误
- 对数学、逻辑推理任务，准确率提升 30%-50%
- 代价：输出 token 数增加（推理过程也要付费）

## 实验 3：Self-Consistency（自洽性投票）

生成多个答案，投票选择最常见的结果。

In [ ]:
from openai import OpenAI
from collections import Counter
import re

client = OpenAI()

def self_consistency(question: str, n: int = 5) -> str:
    """
    Self-Consistency: 生成多个答案并投票
    """
    prompt = f"{question}\n\n让我们一步步思考。最后用'答案：X'的格式给出最终答案。"
    
    answers = []
    print(f"生成 {n} 个答案...\n")
    
    for i in range(n):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,  # 提高多样性
        )
        
        full_response = response.choices[0].message.content
        
        # 提取最终答案（假设格式为"答案：X"）
        match = re.search(r'答案[：:](.*?)(?:\n|$)', full_response)
        if match:
            final_answer = match.group(1).strip()
        else:
            # 如果没有找到标记，取最后一行
            final_answer = full_response.strip().split('\n')[-1]
        
        answers.append(final_answer)
        print(f"答案 {i+1}: {final_answer}")
    
    # 投票
    counter = Counter(answers)
    most_common = counter.most_common(1)[0]
    
    print(f"\n投票结果：{dict(counter)}")
    print(f"最终答案：{most_common[0]} (出现 {most_common[1]}/{n} 次)")
    
    return most_common[0]

# 测试
question = "一个数乘以 3 再加 7 等于 28，这个数是多少？"
result = self_consistency(question, n=5)

In [ ]:
# 测试更复杂的问题
question2 = """
一个班级有 30 名学生。其中：
- 60% 的学生喜欢数学
- 50% 的学生喜欢英语
- 20% 的学生两门都喜欢

问：有多少学生至少喜欢一门课？
"""

result2 = self_consistency(question2, n=5)

**Self-Consistency 的权衡：**

| 优点 | 缺点 |
|-----|------|
| ✅ 准确率大幅提升（尤其是复杂推理） | ❌ API 成本翻 5-10 倍 |
| ✅ 适合高风险场景（金融计算、医疗诊断） | ❌ 响应时间变慢 |
| ✅ 可以发现模型"犹豫"的情况 | ❌ 不适合开放式问题 |

**适用场景：**
- 选择题、判断题
- 数学计算题
- 需要高准确率的分类任务

## 实验 4：综合对比实验

同一个任务，对比所有技巧的效果。

In [ ]:
from openai import OpenAI

client = OpenAI()

# 测试任务：情感分类（混合情感）
test_case = "价格有点贵，但质量确实不错，总体还算满意。"

print("测试评论：", test_case)
print("="*80)
print()

# 1. Zero-shot
prompt_zero = f"判断评论情感（正面/负面/中性）：{test_case}"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt_zero}],
    temperature=0,
)

print("=== 1. Zero-shot ===")
print(f"结果: {response.choices[0].message.content}")
print(f"Token 用量: {response.usage.total_tokens}")
print()

In [ ]:
# 2. Few-shot
prompt_few = f"""
判断评论情感（正面/负面/中性）。

示例：
评论：这个产品太棒了！
情感：正面

评论：质量太差，非常失望。
情感：负面

评论：还行吧，凑合用。
情感：中性

现在判断：
评论：{test_case}
情感：
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt_few}],
    temperature=0,
)

print("=== 2. Few-shot ===")
print(f"结果: {response.choices[0].message.content}")
print(f"Token 用量: {response.usage.total_tokens}")
print()

In [ ]:
# 3. Chain-of-Thought
prompt_cot = f"""
判断评论情感（正面/负面/中性）。

示例：
评论：这个产品太棒了！
分析：用词"太棒了"表达强烈正面情绪。
情感：正面

评论：质量太差，非常失望。
分析：用词"太差"和"失望"都是负面词汇，没有正面内容。
情感：负面

评论：还行吧，凑合用。
分析：用词"还行"和"凑合"表示勉强接受，不是明确的正面或负面。
情感：中性

现在判断：
评论：{test_case}
分析：
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt_cot}],
    temperature=0,
)

print("=== 3. Chain-of-Thought ===")
print(f"结果:\n{response.choices[0].message.content}")
print(f"Token 用量: {response.usage.total_tokens}")
print()

## 动手练习

1. **测试 Few-shot 的示例数量**：对比 2 个、3 个、5 个示例的效果差异
2. **CoT 变体**：试试"Let's think step by step"的英文版本，对比中文版本效果
3. **Self-Consistency 参数调优**：试试 n=3、n=5、n=10，观察准确率和成本的权衡
4. **创建自己的 Few-shot 模板**：为"代码 bug 分类"（语法错误/逻辑错误/性能问题）创建 Few-shot 模板

## 关键要点总结

1. **Few-shot**：给 3-5 个示例，适合分类、格式化任务，成本适中
2. **Chain-of-Thought**：加"让我们一步步思考"，数学推理准确率提升 30%-50%
3. **Self-Consistency**：生成 5-10 个答案投票，适合高风险场景，成本翻倍
4. **成本权衡**：进阶技巧会显著增加 token 消耗，根据场景选择
5. **组合使用**：Few-shot + CoT 效果最好，但成本最高

**选择指南：**

| 场景 | 推荐技巧 |
|-----|----------|
| 简单分类 | Few-shot |
| 数学推理 | CoT |
| 高风险决策 | Self-Consistency |
| 复杂推理 | Few-shot + CoT |

---

**下一步：** 学习 [4.3 结构化输出](./structured_output.ipynb)，让 AI 输出可靠的 JSON。